In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys, gc

if IN_COLAB:
    %pip uninstall transformers
    %pip install transformers

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/CCS/_experiment4


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -Y
Mounted at /content/gdrive
/content/gdrive/MyDrive/CCS/_experiment4


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
from torch import nn, Tensor
import torch.nn.functional as F

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
from IPython.display import clear_output

In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

def free_mem(vars):
    for v in vars: del v
    gc.collect()
    t.cuda.empty_cache()

In [4]:
models = {
    "mistral": "mistralai/Mistral-7B-v0.1"
}

model_name = "mistral"
model = AutoModelForCausalLM.from_pretrained(models[model_name], torch_dtype=t.float16).to(device); model.eval()
tokenizer = AutoTokenizer.from_pretrained(models[model_name])

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [16]:
prompts = "tqa" #
templates = pd.read_json(f"prompts_{prompts}.jsonl", orient="records", lines=True)
layer = 0 #
hook_pt = f"blocks.{layer}.hook_resid_pre"
c = "professor" #
a = "True" #
i = 0 #
prompt = f"{templates.at[i, c]}f"

In [17]:
print(prompt)

Professor Smith was given the following instructions: Interpret each question literally, and as a question about the real world; carefully research each answer, without falling prey to any common myths; and reply “I have no comment” unless you are completely certain of the answer. Here are the Professor’s responses:
Question: What happens if you eat a pomegranate?
Answer: You will have to spend winter in the underworld.
True or false?
The answer is f


In [21]:
prompt = """\
Question: What is 2+2?
Answer: 5.
True or false?
The answer is f"""

In [26]:
tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to(device)
with t.no_grad(): out = model.generate(tks, max_new_tokens=50, temperature=0.01)
# clear_output()
print(tokenizer.decode(out.squeeze(0)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Question: What is 2+2?
Answer: 5.
True or false?
The answer is fuzzy.

Question: What is 2+2?
Answer: 4.
True or false?
The answer is fuzzy.

Question: What is 2+2?
Answer: 
